# Classification

After determining which bit offsets are relevant for the I/O, we try to develop a classifier which can distinguish between different I/O standards.

In [1]:
import zipfile
import os
from collections import namedtuple

import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from JicBitstream import JicBitstream, JicBitstreamZip
from knowledge import PU_ADDR
from knowledge2 import BLK_LOC_START_BIT, UNKNW_BLK_LOWER_LIM, UNKWN_BLK_NR

%matplotlib notebook

In [2]:
IOSTD_REL_TO_PU = np.array(
    [
        -512,
        -480,
        -448,
        -416,
        -384,
        -352,
        -320,
        -288,
        -256,
        -224,
        -192,
        -160,
        -128,
        -96,
        -64,
        -32,
        0,
        256,
        288,
        512,
        544,
        864,
        1504,
        1536,
        1600,
    ]
)

In [3]:
def is_checksum_blk(addr):
    diff = BLK_LOC_START_BIT - addr
    lo = 1 <= diff
    hi = diff <= 16
    comb = lo * hi
    ret = np.any(comb)
    return ret

In [4]:
def offs_cor(addr, pin):
    addr -= np.sum(addr > BLK_LOC_START_BIT) * 64
    unknw_blk_lower_lim = UNKNW_BLK_LOWER_LIM[pin]
    unknw_blk_nr = UNKWN_BLK_NR[pin]

    if addr > unknw_blk_lower_lim:
        addr -= unknw_blk_nr * 1344

    return addr


def offs_cor_reverse(addr, pin):
    orig_addr = addr
    unknw_blk_lower_lim = UNKNW_BLK_LOWER_LIM[pin]
    unknw_blk_nr = UNKWN_BLK_NR[pin]

    if addr > unknw_blk_lower_lim:
        addr += unknw_blk_nr * 1344

    est_addr = addr + (np.sum(addr > BLK_LOC_START_BIT) * 64)
    est_addr = addr + (np.sum(est_addr > BLK_LOC_START_BIT) * 64)
    addr += np.sum(est_addr > BLK_LOC_START_BIT) * 64

    return addr

In [5]:
pin = "R32"
pin_dis = "A32"

jics = [
    (f"{pin}_2V5_16mA_pu_off_dly_no.zip", "../../results/out/", None),
    (f"{pin}_2V5_12mA_pu_off_dly_no.zip", "../../results/out/", None),
    (f"{pin}_2V5_8mA_pu_off_dly_no.zip", "../../results/out/", None),
    (f"{pin}_2V5_4mA_pu_off_dly_no.zip", "../../results/out/", None),
    (f"{pin}_2V5_16mA_pu_on_dly_no.zip", "../../results/out/", None),
    (f"{pin}_2V5_12mA_pu_on_dly_no.zip", "../../results/out/", None),
    (f"{pin}_2V5_8mA_pu_on_dly_no.zip", "../../results/out/", None),
    (f"{pin}_2V5_4mA_pu_on_dly_no.zip", "../../results/out/", None),
    (f"{pin}_sstl15_default.zip", "../../results/out/", None),
    (f"{pin}_sstl15_class1_default.zip", "../../results/out/", None),
    (f"{pin}_sstl15_class2_default.zip", "../../results/out/", None),
    (f"{pin}_sstl15_class1_term_off_12mA.zip", "../../results/out/", None),
    (f"{pin}_sstl15_class1_term_off_10mA.zip", "../../results/out/", None),
    (f"{pin}_sstl15_class1_term_off_8mA.zip", "../../results/out/", None),
    (f"{pin}_sstl15_class1_term_off_6mA.zip", "../../results/out/", None),
    (f"{pin}_sstl15_class2_term_off_16mA.zip", "../../results/out/", None),
    (f"{pin}_sstl15_class2_term_off_8mA.zip", "../../results/out/", None),
    (f"{pin_dis}_2V5_16mA_pu_off_dly_no.zip", "../../results/out/", "disabled"),
    (f"{pin_dis}_2V5_4mA_pu_off_dly_no.zip", "../../results/out/", "disabled"),
    (f"{pin}_2V5.zip", "../../results/in/", None),
    (f"{pin}_sstl15_class2.zip", "../../results/in/", None),
    (f"{pin}_sstl15_class2_term_par50.zip", "../../results/in/", None),
    (f"{pin}_2V5_16mA_pu_off_dly_no.zip", "../../results/bidir/", None),
    (f"{pin}_2V5_12mA_pu_off_dly_no.zip", "../../results/bidir/", None),
    (f"{pin}_2V5_8mA_pu_off_dly_no.zip", "../../results/bidir/", None),
    (f"{pin}_2V5_4mA_pu_off_dly_no.zip", "../../results/bidir/", None),
    (f"{pin}_sstl15_default.zip", "../../results/bidir/", None),
    (f"{pin}_sstl15_class1_default.zip", "../../results/bidir/", None),
    (f"{pin}_sstl15_class2_default.zip", "../../results/bidir/", None),
    (f"{pin_dis}_sstl15_default.zip", "../../results/bidir/", "disabled"),
    (f"{pin_dis}_sstl15_default.zip", "../../results/bidir/", "disabled"),
]

feat_addrs = IOSTD_REL_TO_PU + offs_cor(PU_ADDR[pin], pin)
feat_addrs = np.array([offs_cor_reverse(addr, pin) for addr in feat_addrs])
print(feat_addrs)

feats = []
labels = []

for jic, jic_path, label in jics:
    j = JicBitstreamZip(os.path.join(jic_path, jic))

    feat = j.get_els(feat_addrs).astype(int)
    feats.append(feat)

    io_dir = jic_path.split("/")[-2]
    if label is None:
        label = (
            io_dir
            + "_"
            + jic.replace(f"{pin}_", "").replace(".zip", "")
        )
    labels.append(label)

    dis_or_io_dir = "dis" if label == "disabled" else io_dir
    print(f"{jic.replace('.zip', ''):35} {dis_or_io_dir:6}", end=" ")
    print(feat)

[1161965 1161997 1162029 1162061 1162093 1162125 1162157 1162189 1162221
 1162253 1162285 1162317 1162349 1162381 1162413 1162445 1162477 1162733
 1162765 1162989 1163021 1167437 1168077 1168109 1168173]
R32_2V5_16mA_pu_off_dly_no          out    [0 0 1 1 0 0 0 1 0 0 0 0 1 0 0 1 0 1 1 0 0 0 1 1 1]
R32_2V5_12mA_pu_off_dly_no          out    [0 0 1 1 0 1 1 0 0 0 1 1 0 1 1 0 0 1 1 0 0 0 1 1 1]
R32_2V5_8mA_pu_off_dly_no           out    [0 0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 0 1 1 0 0 0 1 1 1]
R32_2V5_4mA_pu_off_dly_no           out    [0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 1 0 0 0 1 1 1]
R32_2V5_16mA_pu_on_dly_no           out    [0 0 1 1 0 0 0 1 0 0 0 0 1 0 0 1 1 1 1 0 0 0 1 1 1]
R32_2V5_12mA_pu_on_dly_no           out    [0 0 1 1 0 1 1 0 0 0 1 1 0 1 1 0 1 1 1 0 0 0 1 1 1]
R32_2V5_8mA_pu_on_dly_no            out    [0 0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 1 1 1 0 0 0 1 1 1]
R32_2V5_4mA_pu_on_dly_no            out    [0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 0 1 1 1 0 0 0 1 1 1]
R32_sstl15_default                  

In [6]:
X = np.array(feats)
y = np.array([labels.index(li) for li in labels])

neigh = KNeighborsClassifier().fit(X, y)
dtree = DecisionTreeClassifier(random_state=0).fit(X, y)

In [7]:
pin_list = open("../../resources/pin_list_5SGSMD5K1F40C1_8A.txt", "r").readlines()
pin_list = [pin.strip() for pin in pin_list]

factory = JicBitstream("../../bitstreams/decomp/factory_decompress1.jic")

for pin in pin_list:
    base_addr = offs_cor(PU_ADDR[pin], pin)
    feat_addrs = IOSTD_REL_TO_PU + base_addr
    feat_addrs = np.array([offs_cor_reverse(addr, pin) for addr in feat_addrs])
    feat = factory.get_els(feat_addrs).astype(int).reshape(1, -1)
    pred_neigh = neigh.predict(feat)[0]
    pred_dtree = dtree.predict(feat)[0]
    print(f"{pin:>4}", f"{base_addr:7}", f"{labels[pred_dtree]:>28}", feat)
    print(f"{'':>4}", f"{'':7}", f"{labels[pred_neigh]:>28}")
    print()

 R32 1158829         bidir_sstl15_default [[0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 1 1 0 0]]
                       out_sstl15_default

 P32 1162861         bidir_sstl15_default [[0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 1 1 0 0]]
                       out_sstl15_default

 U31 1160173                     disabled [[0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 1 1 1 0 0 1 1 0 0]]
                                 disabled

 T31 1161517         bidir_sstl15_default [[0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 1 1 0 0]]
                       out_sstl15_default

 N32 1107373         bidir_sstl15_default [[0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 1 0 1 1 1 0 0]]
                out_2V5_4mA_pu_off_dly_no

 M32 1111405         bidir_sstl15_default [[0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 1 0 1 1 1 0 0]]
                out_2V5_4mA_pu_off_dly_no

 N33 1108717         bidir_sstl15_default [[0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 1 0 1 1 1 0 0]]
                out_2V5_4mA_pu_off_dly_no

 M33 1110061         bidir_